# Optimizing Algorithms

## Mini-batch gradient descent

Training our model through the *entire batch* could not be the most effective method since we have to calculate for every examples, find the average for the cost function, before updating our weights.

A smarter method is to break up the big data in small training set and run a model in parallel (all the other models are running at the same time)

<img src="./images/improv_17.png" alt="Drawing" style="width: 650px;"/>


## Understanding mini-batch gradient descent
The mini-batch gradient descent will have more noise but eventually, it will have a trend.

<img src="./images/improv_18.png" alt="Drawing" style="width: 650px;"/>


There are two extremes that batch could be categorized in:
- We either have the mini-batch size=m, which is the batch gradient descent size
- Or, we could have the mini-batch to be size=1, which will create the stochasitic gradient to train for every example

The drawback with the batch gradient descent is that it will take too long to have the weights updated. Think about it. We will need to find the mean for the cost function for every example and thus we will need to loop through every single example before we continue with our calculations.

If you use stochastic gradient descent, it's cool that you can make progress through every example, the bad thing is that you lose the speed from the vectorization. You will have to perform the calculation per each one.

The best method is something in between where the mini-batch size is not too small nor too big.
- We are able to use vectorization and it will not take too long.

Typical rules:
- If you have a small training set, use the batch gradient descent. 
- Typical mini-batch sizes are 64, 128, 256, 512. This is 2^6, 2^7, 2^8, 2^9.
- Make the mini-batch fit in the CPU/GPU memory

## Exponentially weighted averages

To calculate the exponetiall weighted average, we assign a large weight to the previous periods (i.e. we are assiging a 90/10 split)

- <img src="./images/improv_19.png" alt="Drawing" style="width: 300px;"/>


The following gets a bit complicated:
- <img src="./images/improv_20.png" alt="Drawing" style="width: 550px;"/>
- Noticed that when we increase the beta from the previous period component part of the equation, increases the weight. Hence, we are assuming that the exponetially weighted average aligns with historic data.


## Understanding exponentially weighted averages

Noticed that are using the same formula to update the next formula.

Notice that the graph shows how the formula decays. 
- <img src="./images/improv_22.jpeg" alt="Drawing" style="width: 450px;"/>

It continued to be complicated. So, we see that the 0.9^10 is around .35. Thus, we get that when the beta is .9, we are using about 1/3 of the month or 10 days.

Noticed to compute this formula, we must start with v. Hence, memory is allocated to store this number which is not a lot.


But why should we use the exponetially weighted avg over the weighted avg?
- For one, moving average is highly use in most metrics. Why can't we use this? 
- Think about it. If we use it exponetially, then every value prior has a lesser weight to the entire function. Thus, we are able to weight the more recent data more heavily. 

## Bias correction in exponentially weighted averages

<img src="./images/improv_24.jpeg" alt="Drawing" style="width: 500px;"/>

In the image above, Andrew suggest that during the beginning phase in calculating the weighted averages, we will encounter issues. 
- For example, in period 1, `beta_1` will be 0. Thus, the weighted average will only use period 1 which gets multiplied 0.02. Hence, the exponentially weighted averages will not be an accurate represenation.
- Thus, we calculating get exponentially weighted averages temperature if we start at 0 results in small values during the initial values.

**Solution**

<img src="./images/improv_52.png" alt="Drawing" style="width: 450px;"/>

To solve for the problem during the initial phase, we can use sum the betas ('coefficients') as the denominators. Hence, we will get a more accurate calculation
- The 0.02 (beta_1) represents 2/3 of the summation of the betas.
- Look at the code example below

```python
(0.0196*40)+(0.02*38)
# Output: 1.544

((0.0196*40)+(0.02*38))/(0.0396)
# Output: 38.98989898989899


```

## Gradient Descent with momentum

<img src="./images/improv_27.jpeg" alt="Drawing" style="width: 500px;"/>

**Reminder:** When calculating the partial deritivates of many nodes, the partial derivatives indicates the magnitude of their importance. Hence, we are able to see which node will help decrease the cost function more effectively.
- We must calculate the nudges for each of the nodes in the last layer. 

Using the example from above, we are able to implement what we learned in the beginning. The concept of calculating the exponentially weighted averages for our betas could also be used in our deep layer networks.
- The further we dive into our neural network, the less influence nodes from our earlier portion of our network has.
- **Key idea: we want to move slowly along the y-axis, and quickly among the x-axis. Look at the graph for a better description.**

The intuition is that the current derivative you are looking at, well that's acceleration. For the previous derivatives, they serve as velocity (friction to prevent the gradient descent to run too quick).
- It's very similar to the previous example (for temperature). Here, instead of the previous days, we are performing the exponentially gradient descent against mini-batch. When we use the mini-batches, we might get the a lot of noise. Hence, to prevent that gradient descent to have a lot of noise, we perform the exponetially gradient descent to *normalize* gradient descent.

Looking at the image below, we see that some people will omit the beta in the current derivatives. However, Andre does not use that as its a bit less intutive of the purpose behind the formula.
- <img src="./images/improv_29.jpeg" alt="Drawing" style="width: 450px;"/>


**One thing to remember is that we are not predicting today's temperature but rather finding a way to have a smooth curve!!**

## RMSprop
<img src="./images/improv_30.png" alt="Drawing" style="width: 600px;"/>

- The RMSprop incorporates the a similar concept to the *expontential weighted average*
    - We will continue to use the beta intuition (where $S_{dw}$ and $S_{db}$ is the previous calculation) but we no longer are keeping the derivative but we are squaring the derivative...
    
- Another difference between RMSprop and the Gradient Descent with momentum is that we updated our weighted and bias differently.
    - We will continue to implement the influence from the previous gradients of the mini-batches with the addition of the denonimator
    - Notice the when we update the weight/bias, we are including using the square root to update the weights/bias accordingly.
        - For example, using the image below, we can tell that the slope will be greater in the vertical direction than the horizontal direction. 
            - If the biases, which goes in the vertical direction, will have a larger value in the denominator,  calculates the biases with a small change.
            - If the weights, which goes in the horizontal direction, will have a smalller value in the denominator, calculates the weights with a larger change.
            
            
- RMS mean Root Mean Squared:
    - You are first squaring the derivatives
    - Then, you take the square root
- We can also use RMSprop with momemntum!
    - Thus, we rename the beta as beta_2
    
- **REMEMBER**: Our denominator could be really small. Hence, we need to an epislon just in case. (we cannot have the denominator be 0)

## Adam optimization algorithm
We will combine the RMSprop with SGD with momentum:
- <img src="./images/improv_31.png" alt="Drawing" style="width: 600px;"/>


Notice that we are producing the metrics of the RMSprop and the SDS with momentum.

Hyperparameters choice:
- alpha: needs to be tuned
- beta_1 = 0.9 (dw)
- beta_2 = 0.999 (dw^2)

The ADAM optimization algorithm provides the "exponential weighted average" alongside the nudges in using the partial deritivates from the weights and biases (we go slower in the horizontal axis than the vertical axis)

ADAM: Adaptive Moment Estimation
- Since we are calculating two derivatives, 
    - beta_1: The first moment
    - beta_2: The second moment

## Learning rate decay
During the intial phase of learning, we shoul take long steps (move quicker the minima). As we approach the local minima, we should decrease our learning rate because we do not want to "skip" the local minima.

1 epoch: 1 pass through the data

- $\alpha$ = 1/(1+decay.rate * epoch.num) * (initial_$\alpha)$

Looking at the image below, we can see that as we continue with the epochs, are learning rate ($\alpha$) will decrease/
- <img src="./images/improv_32.png" alt="Drawing" style="width: 450px;"/>


Also, there are other type of decay function:
- <img src="./images/improv_33.png" alt="Drawing" style="width: 450px;"/>

We can also control manually, but this also only works when you are working with small data.

The formula decreases our learning rate as the number of epochs increases

## The problem of local optima
Notice in the image below (left image), we are likely to think we could run into a lot of local optima points...

However, this is not the case. We are more likely to end up in a saddle point like in the image to the right where we are both at a local minima and a local maxima!
- <img src="./images/improv_34.png" alt="Drawing" style="width: 450px;"/>


Local optima are not a huge problem. A bigger problem are plateau because we are going through a small path. 
- <img src="./images/improv_35.png" alt="Drawing" style="width: 450px;"/>